In [33]:
#!/usr/bin/env python3

In [34]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [35]:
"""
Created on Wed Aug 31 10:55:48 2022

@author: shanrahan
"""
#ExtractFromTemmerman.py
################################### README ################################### 
# Code is written to extract data from results of Temmerman and Leschziner 2001.
# Dataset is available at https://turbmodels.larc.nasa.gov/Other_LES_Data/2dhill_periodic.html
###############################################################################

import os
import numpy as np
from matplotlib import pyplot as plt
from scipy.interpolate import griddata, interp2d

# Import Data from Temmerman
os.chdir('/home/unimelb.edu.au/lcampoli/UoM/Testcases/PeriodicHill/01_Data/')

with open('hill_LES_avgresults.dat','r') as f:
    lines = f.readlines()

l =[]
for line in lines[23:]:
    var = np.asarray(line.split())
    l = np.concatenate([l,var.astype(float)])
    
x_ref  = l[:(196*128)]
y_ref  = l[(196*128):2*(196*128)]
p_ref  = l[(2*(196*128)):3*(196*128)]
u_ref  = l[(3*(196*128)):4*(196*128)]
v_ref  = l[(4*(196*128)):5*(196*128)]
w_ref  = l[(5*(196*128)):6*(196*128)]
nu_ref = l[(6*(196*128)):7*(196*128)]
uu_ref = l[(7*(196*128)):8*(196*128)]
vv_ref = l[(8*(196*128)):9*(196*128)]
ww_ref = l[(9*(196*128)):10*(196*128)]
uv_ref = l[(10*(196*128)):11*(196*128)]
vw_ref = l[(11*(196*128)):12*(196*128)]
k_ref  = l[(12*(196*128)):13*(196*128)]

x_ref  = x_ref.reshape((128, 196))
y_ref  = y_ref.reshape((128,196))
u_ref  = u_ref.reshape((128,196))
v_ref  = v_ref.reshape((128,196))
p_ref  = p_ref.reshape((128,196))
nu_ref = nu_ref.reshape((128,196))
uu_ref = uu_ref.reshape((128,196))
uv_ref = uv_ref.reshape((128,196))
vv_ref = vv_ref.reshape((128,196))
k_ref  = k_ref.reshape((128,196))

del(w_ref, ww_ref, vw_ref)

#%% Build Mesh from Cartesian Mesh

xSamp = int(1000)
ySamp = 128
Ymax = 3.035*28
#xGrid = np.linspace(0,252,xSamp)
xGrid = np.zeros([ySamp,xSamp])
yGrid = np.zeros([ySamp,xSamp])
    
    
X = np.linspace(0,126,int(xSamp/2))  #X Coord for bottom wall
Ymin = np.zeros([int(xSamp/2)])         #Y Coord for bottom wall


X1 = np.linspace(0,252, xSamp)


for i in range(int(xSamp/2)):    #Definition of hill on bottom wall
    if X[i] < 9:
        Ymin[i] = min(28, 2.800000000000E+01 + 0.000000000000E+00*X[i] + 6.775070969851E-03*X[i]**2 - 2.124527775800E-03*X[i]**3 )
    if (X[i]>= 9) & (X[i]<14):
        Ymin[i] = 2.507355893131E+01 + 9.754803562315E-01*X[i] - 1.016116352781E-01*X[i]**2  + 1.889794677828E-03*X[i]**3
    if (X[i]>=14) & (X[i]<20):
        Ymin[i] = 2.579601052357E+01 + 8.206693007457E-01*X[i] - 9.055370274339E-02*X[i]**2 + 1.626510569859E-03*X[i]**3
    if (X[i]>=20) & (X[i]< 30):
        Ymin[i] = 4.046435022819E+01 - 1.379581654948E+00*X[i] + 1.945884504128E-02*X[i]**2 - 2.070318932190E-04*X[i]**3
    if (X[i]>=30) & (X[i]<40):
        Ymin[i] = 1.792461334664E+01 + 8.743920332081E-01*X[i] - 5.567361123058E-02*X[i]**2  + 6.277731764683E-04*X[i]**3    
    if (X[i]>=40) & (X[i]<54):
        Ymin[i] = max(0., 5.639011190988E+01 - 2.010520359035E+00*X[i] + 1.644919857549E-02*X[i]**2 + 2.674976141766E-05*X[i]**3 )
        
    
Ymin = np.concatenate((Ymin, np.flip(Ymin)), axis =0)/28.0 # Building next hill

#scale grid stretching to new coords
profile = y_ref[:,0]

ytemp = np.zeros((profile.size))

for i in range(int(xSamp)):
    for j in range(profile.size):
        ytemp[j] = Ymin[i] + (3.035 - Ymin[i])* ( profile[j] - profile.min())/(profile.max() - profile.min())
    yGrid[:,i] = ytemp
    xGrid[:,i] = X1[i]/28.0

#plt.scatter(xGrid, yGrid)    
#plt.show()
#plt.close()

#np.savetxt('tmp.txt', xGrid, newline='\n')

#print(xGrid[0,0])
#print(xGrid[0,200])
#print(xGrid[0,400])
#print(xGrid[0,600])
#print(xGrid[0,800])
#print(xGrid[0,925])
#print(xGrid[0,944])
#print(xGrid[0,962])
#print(yGrid[0,925])
#print(yGrid[0,944])
#print(yGrid[0,962])

# Interpolate data onto Cartesian Mesh
uGrid  = griddata((x_ref.flatten(),y_ref.flatten()),u_ref.flatten(),(xGrid,yGrid),method='linear')
vGrid  = griddata((x_ref.flatten(),y_ref.flatten()),v_ref.flatten(),(xGrid,yGrid),method='linear')
pGrid  = griddata((x_ref.flatten(),y_ref.flatten()),p_ref.flatten(),(xGrid,yGrid),method='linear')
uuGrid = griddata((x_ref.flatten(),y_ref.flatten()),uu_ref.flatten(),(xGrid,yGrid),method='linear')
uvGrid = griddata((x_ref.flatten(),y_ref.flatten()),uv_ref.flatten(),(xGrid,yGrid),method='linear')
vvGrid = griddata((x_ref.flatten(),y_ref.flatten()),vv_ref.flatten(),(xGrid,yGrid),method='linear')
kGrid  = griddata((x_ref.flatten(),y_ref.flatten()),k_ref.flatten(),(xGrid,yGrid),method='linear')

#Pad Nan values with zero
#uGrid[0,:] = 0

0.0
1.8018018018018016
3.603603603603603
5.405405405405404
7.207207207207206
8.333333333333332
8.504504504504505
8.666666666666666
0.7187665028662583
0.8601726225117662
0.9592504113065755


In [36]:
os.chdir('/home/unimelb.edu.au/lcampoli/UoM/Testcases/PeriodicHill/01_Data/Re_10595')

In [37]:
# Extract Profiles Required - Formatted to match existing databases
p0 = np.concatenate([yGrid[:,0, np.newaxis], uGrid[:,0, np.newaxis], vGrid[:,0, np.newaxis], uuGrid[:,0, np.newaxis], vvGrid[:,0, np.newaxis], uvGrid[:,0, np.newaxis], kGrid[:,0, np.newaxis]], axis = 1)
p0 = np.nan_to_num(p0, nan = 0)
np.save('p00.npy', p0)
np.savetxt('p00.txt', p0, delimiter=',')

In [38]:
p11 = np.concatenate([yGrid[:,925, np.newaxis], uGrid[:,925, np.newaxis], vGrid[:,925, np.newaxis], uuGrid[:,925, np.newaxis], vvGrid[:,925, np.newaxis], uvGrid[:,925, np.newaxis], kGrid[:,925, np.newaxis]], axis = 1)
p11 = np.nan_to_num(p11, nan = 0)
np.save('p11.npy', p11)
np.savetxt('p11.txt', p11, delimiter=',')

In [39]:
p12 = np.concatenate([yGrid[:,944, np.newaxis], uGrid[:,944, np.newaxis], vGrid[:,944, np.newaxis], uuGrid[:,944, np.newaxis], vvGrid[:,944, np.newaxis], uvGrid[:,944, np.newaxis], kGrid[:,944, np.newaxis]],  axis = 1)
p12 = np.nan_to_num(p12, nan = 0)
np.save('p12.npy', p12)
np.savetxt('p112.txt', p12, delimiter=',')

In [40]:
p13 = np.concatenate([yGrid[:,962, np.newaxis], uGrid[:,962, np.newaxis], vGrid[:,962, np.newaxis], uuGrid[:,962, np.newaxis], vvGrid[:,962, np.newaxis], uvGrid[:,962, np.newaxis], kGrid[:,962, np.newaxis]], axis = 1)
p13 = np.nan_to_num(p13, nan = 0)
np.save('p13.npy', p13)
np.savetxt('p13.txt', p13, delimiter=',')

In [41]:
p06 = np.concatenate([yGrid[:,6, np.newaxis], uGrid[:,6, np.newaxis], vGrid[:,6, np.newaxis], uuGrid[:,6, np.newaxis], vvGrid[:,6, np.newaxis], uvGrid[:,6, np.newaxis], kGrid[:,6, np.newaxis]], axis = 1)
p06 = np.nan_to_num(p06, nan = 0)
np.save('p06.npy', p06)
np.savetxt('p06.txt', p06, delimiter=',')

In [42]:
i = 111
p111 = np.concatenate([yGrid[:,i, np.newaxis], uGrid[:,i, np.newaxis], vGrid[:,i, np.newaxis], uuGrid[:,i, np.newaxis], vvGrid[:,i, np.newaxis], uvGrid[:,i, np.newaxis], kGrid[:,i, np.newaxis]], axis = 1)
p111 = np.nan_to_num(p111, nan = 0)
np.save('p111.npy', p111)
np.savetxt('p111.txt', p111, delimiter=',')

In [43]:
i = 222
p222 = np.concatenate([yGrid[:,i, np.newaxis], uGrid[:,i, np.newaxis], vGrid[:,i, np.newaxis], uuGrid[:,i, np.newaxis], vvGrid[:,i, np.newaxis], uvGrid[:,i, np.newaxis], kGrid[:,i, np.newaxis]], axis = 1)
p222 = np.nan_to_num(p222, nan = 0)
np.save('p222.npy', p222)
np.savetxt('p222.txt', p222, delimiter=',')

In [44]:
i = 333
p333 = np.concatenate([yGrid[:,i, np.newaxis], uGrid[:,i, np.newaxis], vGrid[:,i, np.newaxis], uuGrid[:,i, np.newaxis], vvGrid[:,i, np.newaxis], uvGrid[:,i, np.newaxis], kGrid[:,i, np.newaxis]], axis = 1)
p333 = np.nan_to_num(p333, nan = 0)
np.save('p333.npy', p333)
np.savetxt('p333.txt', p333, delimiter=',')

In [45]:
i = 444
p444 = np.concatenate([yGrid[:,i, np.newaxis], uGrid[:,i, np.newaxis], vGrid[:,i, np.newaxis], uuGrid[:,i, np.newaxis], vvGrid[:,i, np.newaxis], uvGrid[:,i, np.newaxis], kGrid[:,i, np.newaxis]], axis = 1)
p444 = np.nan_to_num(p444, nan = 0)
np.save('p444.npy', p444)
np.savetxt('p444.txt', p444, delimiter=',')

In [46]:
i = 555
p555 = np.concatenate([yGrid[:,i, np.newaxis], uGrid[:,i, np.newaxis], vGrid[:,i, np.newaxis], uuGrid[:,i, np.newaxis], vvGrid[:,i, np.newaxis], uvGrid[:,i, np.newaxis], kGrid[:,i, np.newaxis]], axis = 1)
p555 = np.nan_to_num(p555, nan = 0)
np.save('p555.npy', p555)
np.savetxt('p555.txt', p555, delimiter=',')

In [47]:
i = 666
p666 = np.concatenate([yGrid[:,i, np.newaxis], uGrid[:,i, np.newaxis], vGrid[:,i, np.newaxis], uuGrid[:,i, np.newaxis], vvGrid[:,i, np.newaxis], uvGrid[:,i, np.newaxis], kGrid[:,i, np.newaxis]], axis = 1)
p666 = np.nan_to_num(p666, nan = 0)
np.save('p666.npy', p666)
np.savetxt('p666.txt', p666, delimiter=',')

In [48]:
i = 777
p777 = np.concatenate([yGrid[:,i, np.newaxis], uGrid[:,i, np.newaxis], vGrid[:,i, np.newaxis], uuGrid[:,i, np.newaxis], vvGrid[:,i, np.newaxis], uvGrid[:,i, np.newaxis], kGrid[:,i, np.newaxis]], axis = 1)
p777 = np.nan_to_num(p777, nan = 0)
np.save('p777.npy', p777)
np.savetxt('p777.txt', p777, delimiter=',')

In [49]:
i = 888
p888 = np.concatenate([yGrid[:,i, np.newaxis], uGrid[:,i, np.newaxis], vGrid[:,i, np.newaxis], uuGrid[:,i, np.newaxis], vvGrid[:,i, np.newaxis], uvGrid[:,i, np.newaxis], kGrid[:,i, np.newaxis]], axis = 1)
p888 = np.nan_to_num(p888, nan = 0)
np.save('p888.npy', p888)
np.savetxt('p888.txt', p888, delimiter=',')